In [1]:
%pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


In [8]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer

dataset = load_dataset("opus_books", "en-ru")  # или другие пары
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [19]:
def preprocess_ru_to_en(examples):
    inputs = [ex["ru"] for ex in examples["translation"]]
    targets = [ex["en"] for ex in examples["translation"]]
    # Токенизация входов
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    # Токенизация целей
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [22]:
tokenized_datasets = dataset.map(preprocess_ru_to_en, batched=True)

In [59]:
def get_text_lengths(example):
    return {"length": len(example["translation"]["ru"].split())}  # Длина текста на русском

lengths_dataset = dataset["train"].map(get_text_lengths)
print(max(lengths_dataset["length"]))  # Вывод первых 10 длин

150


In [21]:
from transformers import MarianConfig, MarianMTModel

# Конфигурация новой модели с нуля (русский → английский)
config = MarianConfig(
    vocab_size=tokenizer.vocab_size,
    encoder_layers=6,
    decoder_layers=6,
    d_model=512,
    decoder_attention_heads=8,
    encoder_attention_heads=8
)

model = MarianMTModel(config)


In [36]:
train_test_split = tokenized_datasets["train"].select(range(500)).train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

In [37]:
from transformers import DataCollatorForSeq2Seq, Trainer, TrainingArguments

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = TrainingArguments(
    output_dir="./mt_model_ru_en",
    evaluation_strategy="epoch",
    report_to="none",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    logging_dir="./logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

<ipython-input-37-7c9c27d9e4eb>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [60]:
train_test_split = tokenized_datasets["train"].select(range(500, 1000)).train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

In [63]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,1.668718
2,No log,1.557911
3,No log,1.524218


TrainOutput(global_step=87, training_loss=1.8847665019419002, metrics={'train_runtime': 2519.9977, 'train_samples_per_second': 0.536, 'train_steps_per_second': 0.035, 'total_flos': 71880199372800.0, 'train_loss': 1.8847665019419002, 'epoch': 3.0})

In [64]:
# Сохраняем модель
trainer.save_model("ru_to_en_transformer")

# Сохраняем токенизатор (важно для декодирования/генерации)
tokenizer.save_pretrained("ru_to_en_transformer")

('ru_to_en_transformer/tokenizer_config.json',
 'ru_to_en_transformer/special_tokens_map.json',
 'ru_to_en_transformer/vocab.json',
 'ru_to_en_transformer/source.spm',
 'ru_to_en_transformer/target.spm',
 'ru_to_en_transformer/added_tokens.json')

In [53]:
print(tokenizer.decode(train_dataset[3]["labels"]))

There's no joy without...'</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


In [67]:
text = "Пока"

# Токенизация входного текста
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Генерация перевода
outputs = model.generate(**inputs)

# Декодируем результат
translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Перевод:", translated_text)

Перевод: Bangkok ' '
